In [ ]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
from parameters import *
from prompt import *
dataset_train = load_dataset('glue', DATASET, split='train')
dataset_val = load_dataset('glue', DATASET, split='validation')
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
def encode(examples):
    if DATASET=='mnli':
        return tokenizer(examples['premise'], examples['hypothesis'],  truncation=True, padding='max_length',max_length=128)
    if DATASET=='qnli':
        return tokenizer(examples['question'], examples['sentence'],  truncation=True, padding='max_length',max_length=128)
    if DATASET=='qqp': 
        return tokenizer(examples['question1'], examples['question2'], truncation=True, padding='max_length',max_length=128)
    if DATASET=='sst2': 
        return tokenizer(examples['sentence'],  truncation=True, padding='max_length',max_length=128)

dataset_train = dataset_train.map(encode, batched=True)
dataset_val = dataset_val.map(encode, batched=True)

dataset_train = dataset_train.map(lambda examples: {'labels': examples['label']}, batched=True)
dataset_val = dataset_val.map(lambda examples: {'labels': examples['label']}, batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=1)
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=CLIENT_NUM)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,AutoConfig


config=AutoConfig.from_pretrained(MODEL)
model = BertPrefixForSequenceClassification.from_pretrained(MODEL) #BertPromptForSequenceClassification/BertForSequenceClassification

from tqdm import tqdm 
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.train().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=LR)

for name,para in model.named_parameters():
    if 'prefix_encoder' not in name and 'classifier' not in name  and 'pooler' not in name :
        para.requires_grad=False
    else:
        para.requires_grad=True


In [ ]:
from tqdm import tqdm
from sklearn.metrics import *
import numpy as np
norm_l2=NORMCLIP
eps=10.
delta=1e-3  
all_loss=0.
for epoch in range(EPOCH):
    last_update=model.state_dict()
    current_update=[]
    for i, batch in enumerate(tqdm(dataloader_train)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        all_loss+=loss.data
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), NORMCLIP)
        optimizer.step()
        optimizer.zero_grad()
        current_update.append(model.state_dict())
        model.load_state_dict(last_update)
        if (i+1)%CLIENT_NUM==0:
            new_state={x:torch.mean(torch.stack([update[x] for update in current_update],dim=0),dim=0) if (last_update[x].type is torch.float) else last_update[x] for x in last_update}
            for name,p in model.named_parameters():
                if p.grad is not None:
                    # add equivalent noise after aggregation
                    new_state[name] += torch.FloatTensor(np.random.normal(0, LR*2*NORMCLIP*np.sqrt(2*np.log(1.25/DELTA))/EPS/np.sqrt(CLIENT_NUM),size=p.grad.size())).to(device)
            model.load_state_dict(new_state)
            last_update=model.state_dict()
            current_update=[] 
            
        if i % (10*CLIENT_NUM) == 0:
            print(f"loss: {all_loss/(i+1)}")
    model.eval()  
    all_pred=[]
    all_label=[]
    for i, batch in enumerate(tqdm(dataloader_val)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        all_pred += np.argmax(outputs[1].detach().cpu().numpy(),axis=-1).tolist()
        all_label+=batch['labels'].detach().cpu().numpy().tolist() 
    print(accuracy_score(all_label,all_pred))
    model.train()
        